## Split Image Tool

---

### Requirements
- Cut the target image into slices of 416x416
- Mark the start pos of the slice in its filename
- Generate groundtruth (labels) correspond to each slice
    - If the centre point is in the slice, then mark the object to be in the slice
- Cut with an overlap of 20%

In [80]:
IMG_FOLDER = "E:/COMP8500/4K-IMG/"
TARGET_FOLDER = "E:/COMP8500/Sliced-4K/"
SLICE = 416
OVERLAP = 0.2
STEP = int(SLICE * (1 - OVERLAP))

from PIL import Image
import sys

### Define how to split the image
- Need to provide the name of the image, without suffix
- Will cut split the images with the preset settings\
- Only keep the slices that have object in it

In [81]:
def splitImage(imageName, isTrainingSample=False):
    # Read in the image
    image = Image.open(IMG_FOLDER + imageName + ".jpg")
    (img_width, img_height) = image.size
    if isTrainingSample:
        # Read in the groundtruth file
        txt = open(IMG_FOLDER + imageName + ".txt")
        contents = txt.readlines()
        txt.close()
    # Prepare to store the target slices
    slice_list = []
    x_pos = 0
    while x_pos < img_width:
        y_pos = 0
        while y_pos < img_height:
            # when the row of last slice went out of the image
            if x_pos + SLICE > img_width:
                if y_pos + SLICE <= img_height:
                    cur_slice = (img_width - SLICE, y_pos, img_width, y_pos + SLICE)
                # all out of bounds
                else:
                    cur_slice = (img_width - SLICE, img_height - SLICE, img_width, img_height)
            else:
                if y_pos + SLICE <= img_height:
                    cur_slice = (x_pos, y_pos, x_pos + SLICE, y_pos + SLICE)
                # the col went out of the image
                else:
                    cur_slice = (x_pos, img_height - SLICE, x_pos + SLICE, img_height)
            # If not training sample, all slices will be kept (without groundtruth file)
            if not isTrainingSample:
                slice_list.append(cur_slice)
            # Only keep the slices that have target objects
            elif labelSlice(cur_slice, imageName, img_width, img_height, contents):
                slice_list.append(cur_slice)
            y_pos += STEP
        x_pos += STEP
    slices = [image.crop(cur_slice) for cur_slice in slice_list]
    # Save the images
    for idx in range(len(slice_list)):
        slices[idx].save(TARGET_FOLDER + imageName + "." + str(slice_list[idx][0]) + "." + str(slice_list[idx][1]) + ".jpg")

### Check if the slice contains any object
- True: have object(s) in the slice
- If true, the label of the slice will be automatically generated 
    - The format: x_center, y_center, width, height
    - Nomarlised with the size of the slice
    - The width and the height of the bounding box is not changed (no matter if it goes out of the slice)

In [82]:
def labelSlice(cur_slice, imageName, img_width, img_height, contents):
    selected = (cur_slice[0], cur_slice[1])
    new_label_list = []
    # Find if there exists any object in the slice
    for lab_target in contents:
        target = lab_target.split(' ')
        target[1] = img_width * float(target[1])
        target[2] = img_height * float(target[2])
        target[3] = img_width * float(target[3])
        target[4] = img_height * float(target[4])
        # Check if the center point in the slice
        if selected[0] <= target[1] and target[1] <= selected[0] + SLICE:
            if selected[1] <= target[2] and target[2] <= selected[1] + SLICE:
                # Rewrite the position for the current slice
                new_label = [target[0], 0, 0, 0, 0]
                new_label[1] = (target[1] - selected[0]) / SLICE
                new_label[2] = (target[2] - selected[1]) / SLICE
                new_label[3] = target[3] / SLICE
                new_label[4] = target[4] / SLICE
                new_label_list.append(new_label)
    if len(new_label_list) > 0:
        tar_file = open(TARGET_FOLDER + imageName + "." + str(selected[0]) + "." + str(selected[1]) + ".txt", mode='w')
        for new_label in new_label_list:
            out_str = new_label[0] + ", " + str(new_label[1]) + ", " + str(new_label[2]) + ", " + str(new_label[3]) + ", " + str(new_label[4]) + "\n"
            tar_file.write(out_str)
        tar_file.close()
        return True
    else:
        return False

### How to use
- Change the presettings corresponding to ur special requirements
- Create a .txt file to mark all the images you want to split
- Use a loop to call the splitImage Method
    - If it is training data (already labeled with YOLO format), set `isTrainingSample` to be `True`
    - Else leave it blank, all the slices will be kept for testing

In [84]:
IMG = "169.6"
splitImage(IMG,True)